In [1]:
import sys
import os
import subprocess
import tarfile
import shutil

import torch

import boda

boda_src = os.path.join( os.path.dirname( os.path.dirname( os.getcwd() ) ), 'src' )
sys.path.insert(0, boda_src)

from main import unpack_artifact, model_fn

In [2]:
unpack_artifact('gs://syrgoth/aip_ui_test/model_artifacts__20210515_230601__116249.tar.gz')

model_dir = './artifacts'

my_model = model_fn(model_dir)

Loaded model from 20210515_230601


In [3]:
my_model

BassetVL(
  (pad1): ConstantPad1d(padding=[9, 9], value=0.0)
  (conv1): Conv1dNorm(
    (conv): Conv1d(4, 300, kernel_size=(19,), stride=(1,))
    (bn_layer): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pad2): ConstantPad1d(padding=[5, 5], value=0.0)
  (conv2): Conv1dNorm(
    (conv): Conv1d(300, 200, kernel_size=(11,), stride=(1,))
    (bn_layer): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pad3): ConstantPad1d(padding=[3, 3], value=0.0)
  (conv3): Conv1dNorm(
    (conv): Conv1d(200, 200, kernel_size=(7,), stride=(1,))
    (bn_layer): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pad4): ConstantPad1d(padding=(1, 1), value=0.0)
  (maxpool_3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (maxpool_4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (linear1): LinearNorm(
    (linear): Linear(in_featu

In [4]:
my_model(torch.randn(10,4,600))

tensor([[-0.0318, -0.1939, -0.1011],
        [ 0.2486,  0.0610,  0.0991],
        [ 1.4509,  1.1929,  1.0953],
        [ 0.5921,  0.3522,  0.1272],
        [ 0.0997,  0.1604,  0.0276],
        [ 0.1442,  0.1161,  0.1263],
        [ 1.4222,  1.1104,  1.1020],
        [ 0.0826, -0.1162,  0.0049],
        [ 1.0682,  0.8381,  0.5765],
        [ 0.0326,  0.0901,  0.3959]], grad_fn=<AddmmBackward>)